# Flag Project!!

## TODO
- Translation logic and methods
- Country names -> Language -> Language Codes
- Clean and modularize more

### Installing packages
 googletrans package for translation methods  
 pycountry to help match country name and language code. Hopefully.

In [ ]:
!pip install https://files.pythonhosted.org/packages/19/3d/4e3a1609bf52f2f7b00436cc751eb977e27040665dde2bd57e7152989672/googletrans-3.1.0a0.tar.gz
!pip install pycountry -q -U

In [ ]:
from email.mime.text import MIMEText
from googletrans import Translator
import requests
from requests import get
import re
import sys
import os
import ast
import re
import urllib
import smtplib

### Setting up URL

In [ ]:
url = "https://www.gov.br/mre/pt-br/assuntos/representacoes/estrangeiras-no-brasil"

### Getting Embassies Webpage

In [ ]:
print("Crawling URL %s" % url)
try:
    response = requests.get(url)
except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
    pass

In [ ]:
#This reges matches all Embassies and Emails
emails_names = (re.findall(r"[a-z0-9+_][a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z0-9\.\-+_]+|Embaixada d[a-z0-9^A-zÀ-ú\-+_\s]+|Consulado[\-\s][a-z0-9^A-zÀ-ú\-+_\s]+", response.text, re.I))

### Cleaning email and embassies name list.

In [ ]:
#removes duplicates names and emails
mylist = list(dict.fromkeys(emails_names))

In [ ]:
is_email = True #Last one was an email?
for item in mylist:
  if re.search(r"Embaixada d[a-z0-9^A-zÀ-ú\-+_\s]+|Consulado[a-z0-9^A-zÀ-ú\-+_\s]+", item) and is_email == True: #Item is an embassy and the last one is an email.
    is_email = False
  elif re.search(r"[a-z0-9+_][a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z0-9\.\-+_]+", item): #Item is an email.
    is_email = True
  else: #Is either an repeated name or something else.
    mylist.remove(item)

In [ ]:
mylist.remove(mylist[0])
mylist.remove(mylist[0])
mylist[0] = "Embaixada do Afeganistão no Brasil"

### Reading personal info file

In [54]:
#Reading personal info file
file = open("not_passwords", "r")
contents = file.read()
credentials = ast.literal_eval(contents)
file.close()

## Setting up translation, email config and mailing methods

### Methods

In [81]:
def Translate_body(body, language):
  #This should get the body, translate it using the language code and append to the original text
  translator = Translator()
  text = translator.translate(body, dest=language, src='pt')
  text = body + "\n\n" + text + "\n\n"

def Generate_email(embassy: str, emails: list):
  #Email body and email ending with shipping information.
  email_end = credentials["address"].replace("\\n", "\n")
  body = str("""
  Olá <0>, espero que estejam todos bem.
  Meu nome é Thiago Casa Nova e sou um grande fã de colecionar brindes e bandeiras dos mais diversos lugares que eu puder visitar, contudo por conta da pandemia essa minha paixão teve que ficar quase 2 anos parada. 
  Estou entrando em contato para saber se seria possível me enviar qualquer tipo de brinde relacionado ao seu país, como uma bandeira por exemplo, não precisa ser nada super chique ou caro, uma simples souvenier já será muito agradecida.
  Já me extendi demais, espero não ter atrapalhado, vou deixar minhas informações de endereço abaixo caso seja possível realizar o envio, no mais desejo um ótimo final de semana para todos!

  <1>
  """).replace("<0>", embassy) 



  
  #Translate_body(body, lang) #TODO Translation method

  body = body.replace("<1>", email_end)

  message = MIMEText(body, 'plain', 'utf-8')
  message['Subject'] = 'Pedido de Colecionador'
  dest = str(','.join(emails))
  message['To'] = dest

  Send_emails(message)

def Send_emails(message):
  gmail_user = credentials["email"]
  gmail_password = credentials["password"]
  message['From'] = gmail_user
  
  try:
    smtp = smtplib.SMTP("smtp.gmail.com:587")
    smtp.ehlo()
    smtp.starttls()
    smtp.login(gmail_user, gmail_password)

    smtp.sendmail(message['From'], message['To'], message.as_string())
    print('Send mails to -> ', embassy, message['To'])
  except:
    print("Something went wrong while sending email to: ", message['To'])

### Mailing loop to send email to each Embassy

In [82]:
count = 0
embassy = ""
mail = []
for item in mylist[100:]:
  if re.search(r"[a-z0-9+_][a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z0-9\.\-+_]+", item): #Check if the item is an email
    mail.append(item)
  elif embassy == "":
    embassy = item
  else: #when it hits a non-email item it will send to the embassy and all of it emails.
    Generate_email(embassy, mail)
    mail = []
    embassy = item
    print("emails remaning -> ", count)
    count += 1

Send mails to ->  Consulado-Geral da Áustria em São Paulo brasilia-ob@bmeia.gv.at,consuladosp@austria.org.br,saopaulo@advantageaustria.org
emails remaning ->  0
Send mails to ->  Consulado Honorário da Áustria em Blumenau consulado@austriablumenau.com.br
emails remaning ->  1
Send mails to ->  Consulado Honorário da Áustria em Curitiba c.austria@gritsch.com.br
emails remaning ->  2
Send mails to ->  Consulado Honorário da Áustria em Fortaleza consulaustria@gmail.com,austriaconsul@oi.com.br
emails remaning ->  3
Send mails to ->  Consulado Honorário da Áustria em Porto Alegre kathrinrosen@gmail.com,consuladoaustriapoa@gmail.com
emails remaning ->  4
Send mails to ->  Consulado Honorário da Áustria em Recife atendimento@consuladoaustriarecife.com.br
emails remaning ->  5
Send mails to ->  Consulado Honorário da Áustria no Rio de Janeiro hk-rio@consuladoaustriarj.com.br
emails remaning ->  6
Send mails to ->  Consulado Honorário da Áustria em Salvador consuladoaustriaba@gmail.com
emails r